In [1]:
# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
import pandas as pd
import time
sec_sleep = 0.5

In [2]:
# credentials
email = 'joseph.barbierdarnal@gmail.com'
with open('../credentials.txt', 'r') as file:
    password = file.read()

In [3]:
def scroll_to_bottom(sec_sleep=sec_sleep):
    time.sleep(sec_sleep*10)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".jobs-search-results-list")))
    div_element = driver.find_element(By.CSS_SELECTOR, ".jobs-search-results-list")
    driver.execute_script("arguments[0].scrollIntoView(true);", div_element)
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", div_element)

In [5]:
try:
    driver.quit()
except:
    pass
driver = webdriver.Firefox()
driver.get("https://www.linkedin.com")
time.sleep(sec_sleep)

# accept cookies
cookies_xpath = '//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]'
cookies_button = driver.find_element(By.XPATH, cookies_xpath)
cookies_button.click()
time.sleep(sec_sleep)

# enter mail
login_name_xpath = '//*[@id="session_key"]'
email_field = driver.find_element(By.XPATH, login_name_xpath)
email_field.send_keys(email)
time.sleep(sec_sleep)

# enter password
login_password_xpath = '//*[@id="session_password"]'
password_field = driver.find_element(By.XPATH, login_password_xpath)
password_field.send_keys(password)
time.sleep(sec_sleep)

# click on sign in
signin_button_xpath = '//*[@id="main-content"]/section[1]/div/div/form/div[2]/button'
button_signin = driver.find_element(By.XPATH, signin_button_xpath)
button_signin.click()
time.sleep(sec_sleep)

# open jobs section
driver.get("https://www.linkedin.com/jobs")
time.sleep(sec_sleep)

# enter keyword "data" and press "enter"
input_keyword_xpath = '//*[@id="jobs-search-box-keyword-id-ember23"]'
keyword = 'data'
search_field = driver.find_element(By.XPATH, input_keyword_xpath)
search_field.send_keys(keyword)
search_field.send_keys(Keys.ENTER)
time.sleep(sec_sleep)

# close message section
message_xpath = '//*[@id="ember76"]'
message_close = driver.find_element(By.XPATH, message_xpath)
message_close.click()
time.sleep(sec_sleep)

# scroll down in order to ensure all jobs are displayed
scroll_to_bottom()

In [6]:
def get_job_details(driver):
    # Find all job card elements
    job_cards = driver.find_elements(By.CLASS_NAME, 'job-card-container')

    # Initialize lists to store job details
    job_ids = []
    job_titles = []
    companies = []
    locations = []

    # Iterate over job card elements to extract details
    for card in job_cards:
        job_id = card.get_attribute('data-job-id')
        title_element = card.find_element(By.CSS_SELECTOR, '.job-card-container__link.job-card-list__title')
        company_element = card.find_element(By.CSS_SELECTOR, '.job-card-container__primary-description')
        location_element = card.find_element(By.CSS_SELECTOR, '.job-card-container__metadata-wrapper li')

        if job_id:
            job_ids.append(job_id)
            job_titles.append(title_element.text if title_element else 'N/A')
            companies.append(company_element.text if company_element else 'N/A')
            locations.append(location_element.text if location_element else 'N/A')

    # Create a DataFrame
    job_data = pd.DataFrame({
        'Job ID': job_ids,
        'Title': job_titles,
        'Company': companies,
        'Location': locations
    })

    return job_data


In [7]:
job_df = get_job_details(driver)
n_pages = 8
for page in range(2,n_pages):
    
    # go to next page
    next_page_xpath = f'//button[@aria-label="Page {page}"]'
    next_page_button = driver.find_element(By.XPATH, next_page_xpath)
    next_page_button.click()
    
    # wait and scroll down
    scroll_to_bottom()
    
    # get job infos and add them to the df with jobs
    new_job_df = get_job_details(driver)
    job_df = pd.concat([new_job_df, job_df])

In [11]:
job_df = job_df.reset_index(drop=True)
